# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key
# Output File (CSV)
output_data_file = "C:\\Users\\Angiescomputer\\Desktop\\WeatherHome\\cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
data = pd.read_csv("C:\\Users\\Angiescomputer\\Desktop\\WeatherHome\\cities.csv")
cities_dataframe = pd.read_csv("C:\\Users\\Angiescomputer\\Desktop\\WeatherHome\\cities.csv")
print (cities_dataframe.head())


   City_ID       City  Cloudiness Country        Date  Humidity    Lat  \
0        0  ostrovnoy           2      RU  1558378754        72  68.05   
1        1  mahebourg          75      MU  1558378503        74 -20.41   
2        2    qaanaaq          25      GL  1558378755        73  77.48   
3        3     zhuhai           0      CN  1558378755        39  40.71   
4        4  cape town          20      ZA  1558378755        76 -33.93   

      Lng  Max Temp  Wind Speed  
0   39.51     37.50        7.16  
1   57.70     78.80       11.41  
2  -69.36     22.20        2.37  
3  112.04     44.60        4.47  
4   18.42     55.99        8.05  


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

locations =cities_dataframe[["Lat", "Lng"]]
# locations

humidity =cities_dataframe["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
new_types_df = pd.DataFrame(cities_dataframe, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]
print (new_types_df)

            City  Max Temp  Wind Speed  Cloudiness
0      ostrovnoy     37.50        7.16           2
1      mahebourg     78.80       11.41          75
2        qaanaaq     22.20        2.37          25
3         zhuhai     44.60        4.47           0
4      cape town     55.99        8.05          20
..           ...       ...         ...         ...
543       yabrud     80.60        8.05           0
544      paraiso     84.20       16.11           5
545      veraval     61.00        8.05           0
546  novyy urgal     49.74        2.68         100
547      tongren     55.86        2.84           4

[548 rows x 4 columns]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = cities_dataframe

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
print (hotel_df)

     City_ID         City  Cloudiness Country        Date  Humidity    Lat  \
0          0    ostrovnoy           2      RU  1558378754        72  68.05   
1          1    mahebourg          75      MU  1558378503        74 -20.41   
2          2      qaanaaq          25      GL  1558378755        73  77.48   
3          3       zhuhai           0      CN  1558378755        39  40.71   
4          4    cape town          20      ZA  1558378755        76 -33.93   
..       ...          ...         ...     ...         ...       ...    ...   
543      543       yabrud           0      SY  1558378840        32  33.97   
544      544      paraiso           5      MX  1558378898         4  24.01   
545      545      veraval           0      FR  1558378898        62  49.65   
546      546  novyy urgal         100      RU  1558378899        93  51.07   
547      547      tongren           4      CN  1558378899        64  27.72   

        Lng  Max Temp  Wind Speed Hotel Name  
0     39.51     

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))